# Build Your Own IPyWidget
The notebook here shows how to build your own widget using an existing JS library from scratch inside a jupyter notebook. This is not a complete guide and leaves out testing completely, but it is maybe a useful step between an idea and the great [cookiecutter widget project](https://github.com/jupyter-widgets/widget-cookiecutter)

In [1]:
import numpy as np
import base64

def encode_numpy_b64(in_img, rgb=False):
    # type: (np.ndarray) -> str
    """
    Encode numpy arrays as b64 strings
    :param in_img:
    :return:
    >>> encode_numpy_b64(np.eye(2))
    'AAAAAAAA8D8AAAAAAAAAAAAAAAAAAAAAAAAAAAAA8D8='
    """
    if rgb:
        img_bytes = in_img[:, :].astype(np.uint8).tobytes()
    else:
        img_bytes = in_img.astype(np.uint16).tobytes()
    return base64.b64encode(img_bytes).decode()

## Python Side of a Widget
Here we make the python side of a widget where we create the name, module and the relevant fields we might want to change (the bridge between python and JS) called traitlets.

In [2]:
import traitlets as tr
import ipywidgets as widgets
class CornerstoneWidget(widgets.DOMWidget):
    _view_name = tr.Unicode('CornerstoneWidget').tag(sync=True)
    _view_module = tr.Unicode('cs_widget').tag(sync=True)
    _view_module_version = tr.Unicode('0.1.0').tag(sync=True)
    title_field = tr.Unicode('Awesome Widget').tag(sync=True)
    img_bytes = tr.Unicode('AQAAAAAAAAABAAAAAAAAAAEA').tag(sync=True)
    img_width = tr.Int(3).tag(sync=True)
    img_height = tr.Int(3).tag(sync=True)
    img_min = tr.Float(0).tag(sync=True)
    img_max = tr.Float(255).tag(sync=True)
    img_scale = tr.Float(1.0).tag(sync=True)
    img_color = tr.Bool(False).tag(sync=True)
    _tool_state = tr.Unicode('').tag(sync=True)
    _tool_state_counter = tr.Int(0).tag(sync=True)
    _selected_tool = tr.Unicode('').tag(sync=True)
    
    def select_tool(self, tool_name):
        self._selected_tool=tool_name
    
    def get_tool_state(self):
        self._tool_state_counter+=1
        return self._tool_state
    
    def update_image(self, in_image):
        # type: (CornerstoneWidget, np.ndarray) -> None
        """
        Update the image loaded in the widget
        """
        self.img_height = in_image.shape[0]
        self.img_width = in_image.shape[1]
        if len(in_image.shape)==2:
            self.img_min = float(in_image.min())
            self.img_max = float(in_image.max())
            self.img_color = False
            rs_image = (in_image - self.img_min)
            im_range = self.img_max - self.img_min
            MIN_RANGE = 1
            if im_range<MIN_RANGE:
                self.img_max = self.img_min+MIN_RANGE
                im_range = MIN_RANGE
            rs_image*=(2 ** 16 - 1)/im_range
            self.img_bytes = encode_numpy_b64(rs_image)
        else:
            self.img_color=True
            self.img_min = 0
            self.img_max = 255
            self.img_bytes = encode_numpy_b64(in_image.clip(0, 255).astype(np.uint8), rgb=True)
        self.img_scale = 1.0

# JavaScript
The javascript portion starts using the requirejs tools built into jupyter to get our dependencies (<source> tags sometimes work but are hard to translate to a full self-contained project). Notice here we had to create a `shim` for _cornerstone-core_ since it isn't a standard package. Also notice the missing prefix of `http://` and `.js` at the end. RequireJS just does it like this, no idea why.

In [3]:
%%javascript
require.config({
  paths: {
      'cornerstone-core': '//unpkg.com/cornerstone-core@2.2.4/dist/cornerstone.min',
      cornerstoneMath: '//unpkg.com/cornerstone-math@0.1.6/dist/cornerstoneMath.min',
      cornerstoneTools: '//unpkg.com/cornerstone-tools@2.3.9/dist/cornerstoneTools.min'
  },
    shim: {
        'cornerstone-core': {
            exports: 'cornerstone',
            deps: ['jquery']
        }
    }
});

<IPython.core.display.Javascript object>

## The JS WIdget
Here we create the full JS widget including the dependencies we defined before. If there is a problem with the dependency definition, the dependency object will sometimes be loaded as nil/None and so if your widget does not work be sure to check this.

In [4]:
%%javascript
require.undef('cs_widget');
function str2ab(str) {
                var buf = new ArrayBuffer(str.length*2); // 2 bytes for each char
                var bufView = new Uint16Array(buf);
                var index = 0;
                for (var i=0, strLen=str.length; i<strLen; i+=2) {
                    var lower = str.charCodeAt(i);
                    var upper = str.charCodeAt(i+1);
                    bufView[index] = lower + (upper <<8);
                    index++;
                }
                return bufView;
            }
function str2rgb(str) {
                var buf = new ArrayBuffer(str.length); // 1 bytes for each char
                var bufView = new Uint8Array(buf);
                var index = 0;
                for (var i=0, strLen=str.length; i<strLen; i+=2) {
                    var lower = str.charCodeAt(i);
                    bufView[index] = lower;
                    index++;
                }
                return bufView;
            }
// disable the default context menu which is there by default
function disableContextMenu(e) {
    $(e).on('contextmenu', function(e) {
        e.preventDefault();
    });
}
define('cs_widget', ["@jupyter-widgets/base", "cornerstone-core",
                    'cornerstoneMath', 'cornerstoneTools'], 
       function(widgets, cs, cm, ctools) {
            ctools.external.cornerstone = cs;
            ctools.external.cornerstoneMath = cm;
    var CornerstoneWidget = widgets.DOMWidgetView.extend({
        initialize: function() {
            console.log('Initializing');
            this.message = document.createElement('div');
            this.viewer = document.createElement('div');
            disableContextMenu(this.viewer);
            var fv = $(this.viewer);
            fv.width('512px');
            fv.height('512px');
        },
        render: function() {
            console.log('Rendering!');
            // Enable our tools
            this.el.appendChild(this.message);
            this.el.appendChild(this.viewer);
            this.dicom_changed();
            this.message_changed();
            this.model.on('change:img_bytes', this.dicom_changed, this);
            this.model.on('change:img_scale', this.zoom_changed, this);
            this.model.on('change:title_field', this.message_changed, this);
            this.model.on('change:_tool_state', this.update_cs_state, this);
            this.model.on('change:_tool_state_counter', this.save_cs_state, this);
            this.model.on('change:_selected_tool', this.activate_tool, this);
            var my_viewer = this.viewer;
            var my_model = this.model;
            this.viewer.addEventListener('mousedown', 
                                         function(e) {
                var appState = ctools.appState.save([my_viewer]);
                var appStr = JSON.stringify(appState);
                console.log('State is:'+appStr);
                my_model.set('_tool_state', appStr);
                my_model.save_changes();
            }
                                        )
        },
        parse_image: function(imageB64Data, width, height, min_val, max_val, color) {
            var pixelDataAsString = window.atob(imageB64Data, width, height);
            if (color) {
                var pixelData = str2rgb(pixelDataAsString);
            } else {
                var pixelData = str2ab(pixelDataAsString);
            }
            console.log('decoding: '+width+'x'+height+' => '+pixelData.length)
            function getPixelData() {
                return pixelData;
            }
            var maxPixelValue=65535;
            var sizeInBytes=height * width * 2;
            var slope = (max_val-min_val)/65535.0;
            var intercept = min_val;
            if (color) {
                maxPixelValue=255;
                sizeInBytes=height * width * 4;
                slope=1;
                intercept=0;
            }
            var out_arr  = {
                imageId: '',
                minPixelValue: 0,
                maxPixelValue: maxPixelValue,
                slope: slope,
                intercept: intercept,
                getPixelData: getPixelData,
                windowCenter: 0.5 * (max_val + min_val),
                windowWidth: (max_val - min_val),
                rows: height,
                columns: width,
                height: height,
                width: width,
                color: color,
                columnPixelSpacing: 1.0,
                rowPixelSpacing: 1.0,
                sizeInBytes: sizeInBytes
            };
            return out_arr
        },
        
        message_changed: function() {
            this.message.textContent = this.model.get('title_field');
        },
        
        dicom_changed: function() {
            var img_bytes = this.model.get('img_bytes')
            var img_width = this.model.get('img_width')
            var img_height = this.model.get('img_height')
            var img_min = this.model.get('img_min')
            var img_max = this.model.get('img_max')
            var color = this.model.get('img_color')
            var out_img = this.parse_image(img_bytes, img_width, img_height, img_min, img_max, color);
            console.log(out_img);
            cs.enable(this.viewer);
            this.viewport = cs.getDefaultViewportForImage(this.viewer, out_img);
            cs.displayImage(this.viewer, out_img, this.viewport);
            this.setup_tools();
        },
        setup_tools: function() {
            ctools.mouseInput.enable(this.viewer);
            ctools.mouseWheelInput.enable(this.viewer);
            ctools.wwwc.activate(this.viewer, 1); // Left Click
            ctools.pan.activate(this.viewer, 2); // Middle Click
            ctools.zoom.activate(this.viewer, 4); // Right Click
            ctools.zoomWheel.activate(this.viewer); // Mouse Wheel
        },
        _disable_all_tools: function(element) {
            // helper function used by the tool button handlers to disable the active tool
            // before making a new tool active
            ctools.wwwc.deactivate(element, 1);
            ctools.pan.deactivate(element, 2); // 2 is middle mouse button
            ctools.zoom.deactivate(element, 4); // 4 is right mouse button
            ctools.length.deactivate(element, 1);
            ctools.ellipticalRoi.deactivate(element, 1);
            ctools.rectangleRoi.deactivate(element, 1);
            ctools.angle.deactivate(element, 1);
            ctools.highlight.deactivate(element, 1);
            ctools.freehand.deactivate(element, 1);
        },
        activate_tool: function() {
            var tool_name = this.model.get('_selected_tool');
            if (tool_name=='reset') {
                cs.reset(this.viewer);
                this.setup_tools();
            } else {
                this._disable_all_tools(this.viewer);
                if (tool_name=='zoom') {
                    ctools.zoom.activate(this.viewer, 1);
                }
                if (tool_name=='window') {
                    ctools.wwwc.activate(this.viewer, 1);
                }
                if (tool_name=='pan') {
                    ctools.pan.activate(this.viewer, 1);
                    ctools.zoom.activate(this.viewer, 2);
                }
                if (tool_name=='bbox') {
                    ctools.rectangleRoi.enable(this.viewer);
                    ctools.rectangleRoi.activate(this.viewer, 1);
                }
                if (tool_name=='probe') {
                    ctools.probe.enable(this.viewer);
                    ctools.probe.activate(this.viewer, 1);
                }
                if (tool_name=='highlight') {
                    ctools.highlight.enable(this.viewer);
                    ctools.highlight.activate(this.viewer, 1);
                }
            }
            
        },
        save_cs_state: function() {
            var appState = ctools.appState.save([this.viewer]);
            var appStr = JSON.stringify(appState);
            console.log('State is:'+appStr);
            this.model.set('_tool_state', appStr);
            this.model.save_changes();
        },
        update_cs_state: function() {
            var new_state_json = this.model.get('_tool_state');
            console.log('updating state:'+new_state_json+', '+new_state_json.length);
            if (new_state_json.length>1) {
                var appState = JSON.parse(new_state_json)
                ctools.appState.restore(new_state_json);
            }
            this.save_cs_state();
        },
        zoom_changed: function() {
            this.viewport.scale = this.model.get('img_scale');
            cs.setViewport(this.viewer, this.viewport);
        }
    });

    return {
        CornerstoneWidget : CornerstoneWidget
    };
});

<IPython.core.display.Javascript object>

# Make the Widget

In [5]:
cs_view = CornerstoneWidget()
#cs_view.title_field = 'Cornerstone Widget'

## Create Callbacks
Here we make the callbacks for showing the image and zooming

In [6]:
from IPython.display import Javascript, display
import ipywidgets as ipw
size_scroller = ipw.IntSlider(value=128, min=3, max=2048, description='Image Size')
def show_image(cs_obj, img_maker):
    c_wid = size_scroller.value
    cs_obj.update_image(img_maker(c_wid))

def zoom_viewer(cs_obj, zf):
    cs_obj.img_scale+=zf

In [7]:
noisy_img_but = ipw.Button(description='Noisy Image')
noisy_image = lambda x: np.random.uniform(-1000, 1000, size=(x, x))
noisy_img_but.on_click(lambda *args: show_image(cs_view, 
                                                noisy_image))

gradient_img_but = ipw.Button(description='Gradient Image')
gradient_image = lambda x: np.linspace(-1, 1, x*x).reshape((x, x))
gradient_img_but.on_click(lambda *args: show_image(cs_view, 
                                                gradient_image))

half_img_but = ipw.Button(description='Half Image')
half_image = lambda x: np.eye(x)[:x//2]
half_img_but.on_click(lambda *args: show_image(cs_view, half_image))

cgradient_img_but = ipw.Button(description='Color Gradient Image')
def cgradient_image(x): 
    base_img = np.linspace(127, 255, x*x).reshape((x, x)).astype(np.uint8)
    rgb_img = np.stack([base_img, 
                        base_img.T, 
                        np.full_like(base_img, 255), 
                        base_img], -1)
    return rgb_img

cgradient_img_but.on_click(lambda *args: show_image(cs_view, cgradient_image))

zoom_in_but = ipw.Button(description='Zoom In')
zoom_in_but.on_click(lambda *args: zoom_viewer(cs_view, 0.25))
zoom_out_but = ipw.Button(description='Zoom Out')
zoom_out_but.on_click(lambda *args: zoom_viewer(cs_view, -0.25))
show_image(cs_view, gradient_image) # have a default image

# Make a Nice Panel
Here we show the widget with a few buttons to interact with.

In [8]:
ipw.VBox([
    cs_view,
    ipw.VBox([
        size_scroller,
        ipw.HBox([noisy_img_but, half_img_but, gradient_img_but, cgradient_img_but]),
        ipw.HBox([zoom_in_but, zoom_out_but])
    ])
])